In [ ]:
import zarr
from numcodecs import Blosc, Zlib, Shuffle

import matplotlib.pyplot as plt

import numpy as np

%matplotlib widget

In [ ]:
levels = [1, 5, 9]

In [ ]:
fs = 30000
duration = 30
num_channels = 16

sin_f = 5
sin_amp = 100
noise_amp = 10
phase_shift = 0.1

In [ ]:
# make some data
data = np.zeros((fs * duration, num_channels), dtype="int16")

In [ ]:
for ch in range(num_channels):
    data_h = sin_amp * np.sin(2 * np.pi * sin_f * np.arange(fs * duration)/fs + ch*phase_shift)
    data_h += noise_amp * np.random.randn(fs * duration)
    data[:, ch] = data_h.astype("int16")

In [ ]:
plt.figure()
plt.plot(data[:, ::5])

In [ ]:
Blosc?

In [ ]:
# compress
chunk_size = 30000


for lev in levels:
    compressor_blosc = Blosc(cname="zlib", clevel=lev, shuffle=0)
    compressor_zlib = Zlib(lev)
    
    z_blosc = zarr.open(f'../data/zlib_ex_blosc{lev}.zarr', mode='w', shape=data.shape,
                        compressor=compressor_blosc, chunks=(chunk_size, None), dtype='i2')
    z_noblosc = zarr.open(f'../data/zlib_ex_noblosc{lev}.zarr', mode='w', shape=data.shape,
                          compressor=compressor_zlib, chunks=(chunk_size, None), dtype='i2')
    
    z_blosc[:] = data
    z_noblosc[:] = data
    
    cr_blosc = z_blosc.nbytes / z_blosc.nbytes_stored
    cr_noblosc = z_noblosc.nbytes / z_noblosc.nbytes_stored    
    
    print(f"Level {lev} : blosc {cr_blosc} - non-blosc {cr_noblosc}")